In [1]:
import pandas as pd
import os   #These two are useful modules
import sys
import time
import numpy as np #Numerical Python
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


code_dir = os.getcwd()
main_dir = os.path.join(code_dir,"..")
data_dir = os.path.join(main_dir,"data")
results_dir = os.path.join(main_dir,"results")
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    print("Directory %s created !" % results_dir)

In [2]:
data = pd.read_csv(os.path.join(data_dir,'varianze_bitcoin5.txt'), sep=",", header=None)
data.columns = ["Data", "Price"]
data[['Data','Time']]=data.Data.str.split(expand=True)
data[['Hour','Minute','Second']]=data.Time.str.split(':',expand=True).astype(float)
data[['Year','Month','Day']]=data.Data.str.split('-',expand=True).astype(float)
data['time_of_month_in_minute']=data.Day*1440+data.Hour*60+data.Minute+data.Second/60
print(data)

               Data     Price             Time  Hour  Minute     Second  \
0        2021-09-22  43791.70  19:52:35.325845  19.0    52.0  35.325845   
1        2021-09-22  43791.70  19:52:35.751676  19.0    52.0  35.751676   
2        2021-09-22  43791.70  19:52:36.001670  19.0    52.0  36.001670   
3        2021-09-22  43798.71  19:52:36.251682  19.0    52.0  36.251682   
4        2021-09-22  43798.71  19:52:36.487399  19.0    52.0  36.487399   
...             ...       ...              ...   ...     ...        ...   
1338753  2021-09-27  42938.02  01:13:43.133499   1.0    13.0  43.133499   
1338754  2021-09-27  42942.87  01:13:43.411511   1.0    13.0  43.411511   
1338755  2021-09-27  42942.87  01:13:43.725500   1.0    13.0  43.725500   
1338756  2021-09-27  42942.87  01:13:43.999499   1.0    13.0  43.999499   
1338757  2021-09-27  42942.87  01:13:44.289496   1.0    13.0  44.289496   

           Year  Month   Day  time_of_month_in_minute  
0        2021.0    9.0  22.0             32

In [3]:
#Cut datas you need
def cut_datas(every_minutes,long,bins,start_when_day,start_when_time):
    
    h_in,min_in,sec_in=np.asarray(start_when_time.split(':')).astype(float)
    y_in,m_in,d_in=np.asarray(start_when_day.split('-')).astype(float)

    start_minute=d_in*1440+h_in*60+min_in+sec_in/60
    stop_minute=start_minute+(long+1)*every_minutes
    
    ranges=np.arange(start_minute, stop_minute, every_minutes)
    
    temp=data
    temp['range'] = pd.cut(temp.time_of_month_in_minute, ranges, include_lowest=True)

    ax=temp.hist(column='Price', by='range', bins = bins, figsize=(30,20))
    
    prezzi=[]
    bins2=[]
    for axx in ax:
        for x in axx:
            p = x.patches
            xlim=x.get_xlim()
            xx=np.around(np.linspace(xlim[0],xlim[1],bins),1)
            prezzi.append(xx)
            heights = [patch.get_height() for patch in p]
            bins2.append(heights)
    
        
    return prezzi,bins2

In [4]:
def fit_datas(binnes,prezzi):

    def fit_function(x, a, b, c):
        return a*np.exp(-((x-c)/b)**2)

    #array_di_medie=[]
    array_di_medie_nonfit=[]
    array_di_varianze=[]

    y=binnes
    for k in range(len(y)):
        xx=prezzi[k]
        try:
            initial_values=(np.max(y[k]), 30, xx[np.argmax(y[k])])
            dy=np.full(len(xx), initial_values[1])
            pars, covm = curve_fit(fit_function, xx, y[k], initial_values,dy)

            varianza=round(pars[1],1)
            media=round(pars[2],1)


            if media>35000 and varianza<650:
                #array_di_medie.append(media)
                array_di_medie_nonfit.append(np.mean(xx))
                array_di_varianze.append(varianza)

            x1=np.linspace(np.min(xx),np.max(xx),200)
            y1=pars[0]*np.exp(-((x1-pars[2])/pars[1])**2)

        except:
            pass
    return array_di_medie_nonfit, array_di_varianze


In [5]:
def try_strategy(array_di_medie_nonfit,array_di_varianze): 

    def fit_retta(x, a, b):
        return a*x+b
    
    med_var=np.round(np.mean(array_di_varianze),2)
    med_med=np.round(np.mean(array_di_medie_nonfit),2)

    y_var=(array_di_varianze-med_var)/(np.max(array_di_varianze)-med_var)
    y_med=(array_di_medie_nonfit-med_med)/(np.max(array_di_medie_nonfit)-med_med)
                  

    x2=np.linspace(0,len(y_med),len(y_med))
    score=0

    for i in range(len(y_var)-1):
        try:
            if np.abs(y_var[i+1]-y_var[i])>0.4:
                temp_var=y_var[:i+1]
                temp_med=y_med[:i+1]

                initial_values=((y_med[i+1]-y_med[0])/x2[i],np.mean(temp_var))
                pars, covm = curve_fit(fit_retta, x2[:i+1], temp_med, initial_values)

                y_test=pars[0]*x2+pars[1]

                if y_med[i+1]<y_test[i+1]:
                    rel_score=np.mean(y_med[i+2:i+8])-y_med[i+1]
                    score=score+rel_score
                else:
                    rel_score=-np.mean(y_med[i+2:i+8])+y_med[i+1]
                    score=score+rel_score
        except:
            pass

    return score

In [ ]:
info=[]
for l in range(15,241) :
    print(l)
    for d in range (23,27):
        for h in range(0,24,4):
            start_when_day='2021-09-'+str(d)
            start_when_time=str(h)+':00:00'
            every_minutes=1
            long=l
            bins=9
            prezzi,binnes=cut_datas(every_minutes,long,bins,start_when_day,start_when_time)
            mediex,varianzex=fit_datas(binnes,prezzi)
            scorex=try_strategy(mediex,varianzex)
            info.append([scorex,l])

15


<ipython-input-5-bf8af09ebd50>:22: RuntimeWarning: divide by zero encountered in double_scalars
  initial_values=((y_med[i+1]-y_med[0])/x2[i],np.mean(temp_var))
<ipython-input-5-bf8af09ebd50>:4: RuntimeWarning: invalid value encountered in multiply
  return a*x+b
/home/marco/.local/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/marco/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/marco/.local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/marco/.local/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:218: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot 

16
17
18
19


In [ ]:
#plot istogramma degli scores per lunghezza di analisi in minuti

df = pd.DataFrame(info, columns=['Scores', 'minuti'])
bins=10
axel=df.hist(column='Scores', by='minuti', bins = bins, figsize=(30,20))